# Importando módulos 

In [4]:
import obspy

from multiprocessing import Pool
from obspy import read,UTCDateTime,Trace
from obspy.clients.fdsn import Client
import os
import glob
import numpy as np
from collections import defaultdict
import pandas as pd
from pandas import Timestamp, date_range

from tqdm import tqdm


import itertools

#para plotar as figuras
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition,inset_axes
import matplotlib
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

from shapely.geometry.polygon import LinearRing

import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from IPython.display import HTML
from IPython import display

# Inputs e Outputs

In [5]:
FOLDER_OUTPUT = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/'

MSEED_INPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/MSEED_LFI/"

EVENTs_DATA_FOLDER = '/home/sysop/Documents/codes_marefone/EVENT_MSEED/'

SELECT_EVENTs_DATA_FOLDER = '/home/sysop/Documents/codes_marefone/EVENT_MSEED_SEL/'

METADATA_OUTPUT = "/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/metadados_glider_acustico_pmpas-bs.csv"

campanha_csv = '/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/Equipamentos/informacoes_sensores_gliders/Dados das aquisicoes_LFIs.xls'

# Extraindo informações dos arquivos ".mseed"

In [6]:
filenames_MSEED = sorted(glob.glob(MSEED_INPUT+'*/*/*.mseed'))

In [7]:
len(filenames_MSEED)

1892563

In [8]:
lst_amar = []
for i in filenames_MSEED:
    lst_amar.append(i.split('/')[-1].split('.')[0])

In [9]:
lst_amarS = sorted(list(set(lst_amar)))

In [10]:
df_lst = []
for o in lst_amarS:
    k = [j.split('/')[-1] for j in filenames_MSEED if o in j]
    print(k[0],k[-1])

    sensor = o
    if len(k[0].split('.')) == 4:
        starttime = k[0].split('.')[2]
    
    if len(k[-1].split('.')) == 4:
        endtime = k[-1].split('.')[2]
       
    if len(k[0].split('.')) == 3:
        starttime = k[0].split('.')[1]

    if len(k[-1].split('.')) == 3:
        endtime = k[-1].split('.')[1]
    
    starttime_file = k[0]
    endtime_file = k[-1]

    df = pd.DataFrame([[sensor],[starttime_file],[endtime_file],[starttime],[endtime]], index=['sensor', 'starttime_file', 'endtime_file','starttime','endtime']).T
    df_lst.append(df)
    

AMAR021.1.20191211T150000Z.mseed AMAR021.1.20220714T084329Z.mseed
AMAR038.1.20191229T000001Z.mseed AMAR038.1.20220722T201001Z.mseed
AMAR045.1.20200102T225009Z.mseed AMAR045.1.20220729T180001Z.mseed
AMAR096.1.20191213T171001Z.mseed AMAR096.1.20200908T191001Z.mseed
AMAR115.1.20191223T110001Z.mseed AMAR115.1.20220719T192001Z.mseed
AMAR116.1.20191213T182001Z.mseed AMAR116.1.20220718T064001Z.mseed
AMAR117.1.20201011T170002Z.mseed AMAR117.1.20220719T173250Z.mseed
AMAR122.1.20191229T002001Z.mseed AMAR122.1.20220722T044001Z.mseed
AMAR146.1.20191229T000001Z.mseed AMAR146.1.20210430T184001Z.mseed
AMAR147.1.20200103T005001Z.mseed AMAR147.1.20220716T063001Z.mseed
AMAR149.1.20191213T152001Z.mseed AMAR149.1.20220718T081001Z.mseed
AMAR164.1.20191206T150001Z.mseed AMAR164.1.20220802T013001Z.mseed
AMAR165.1.20191221T100001Z.mseed AMAR165.1.20220720T035001Z.mseed
AMAR169.1.20191206T150001Z.mseed AMAR169.1.20200422T172001Z.mseed
AMAR170.20191221T100005Z.mseed AMAR170.1.20220724T015009Z.mseed
AMAR184.1.20

In [11]:
dataframe_files = pd.concat(df_lst, ignore_index=True)
dataframe_files['starttime'] = pd.to_datetime(dataframe_files.starttime, format='%Y%m%dT%H%M%SZ')
dataframe_files['endtime'] = pd.to_datetime(dataframe_files.endtime, format='%Y%m%dT%H%M%SZ')
dataframe_files.sort_values(by=['starttime','sensor'])

,sensor,starttime_file,endtime_file,starttime,endtime
11,AMAR164,AMAR164.1.20191206T150001Z.mseed,AMAR164.1.20220802T013001Z.mseed,2019-12-06 15:00:01,2022-08-02 01:30:01
13,AMAR169,AMAR169.1.20191206T150001Z.mseed,AMAR169.1.20200422T172001Z.mseed,2019-12-06 15:00:01,2020-04-22 17:20:01
16,AMAR187,AMAR187.1.20191206T150001Z.mseed,AMAR187.1.20220801T000001Z.mseed,2019-12-06 15:00:01,2022-08-01 00:00:01
0,AMAR021,AMAR021.1.20191211T150000Z.mseed,AMAR021.1.20220714T084329Z.mseed,2019-12-11 15:00:00,2022-07-14 08:43:29
15,AMAR184,AMAR184.1.20191211T150000Z.mseed,AMAR184.1.20220714T101009Z.mseed,2019-12-11 15:00:00,2022-07-14 10:10:09
10,AMAR149,AMAR149.1.20191213T152001Z.mseed,AMAR149.1.20220718T081001Z.mseed,2019-12-13 15:20:01,2022-07-18 08:10:01
3,AMAR096,AMAR096.1.20191213T171001Z.mseed,AMAR096.1.20200908T191001Z.mseed,2019-12-13 17:10:01,2020-09-08 19:10:01
5,AMAR116,AMAR116.1.20191213T182001Z.mseed,AMAR116.1.20220718T064001Z.mseed,2019-12-13 18:20:01,2022-07-18 06:40:01
12,AMAR165,AMAR165.1.20191221T100001Z.mseed,AMAR165.1.20220720T035001Z.mseed,2019-12-21 10:00:01,2022-07-20 03:50:01
14,AMAR170,AMAR170.20191221T100005Z.mseed,AMAR170.1.20220724T015009Z.mseed,2019-12-21 10:00:05,2022-07-24 01:50:09


# Dados LFI

Em cada linha, foram instalados gravadores acústicos nas profundidades de 50 m, 200 m e 950 m. 

No primeiro ciclo, foram lançadas seis linhas e no segundo, quatro linhas.

|Fundeio|Profundidade(m)|Região|
|---:|:---------|:---------------------------------|
|FA01|2.200|Impactada por atividade E&P|
|FA02|2.000|Em desenvolvimento de atividade E&P|
|FA03|2.850|Região sem atividade E&P|
|FA04|1.100|Região sem atividade E&P|
|FA05|1.100|Rota de navegação de E&P|
|FA06|1.100|Rota de navegação de E&P|





## LINHA DE FUNDEIO INSTRUMENTADA (LFI)

E.1 DESCRIÇÃO DO AMAR-G3

O equipamento AMAR-G3 (Autonomous Multichannel Acoustic Recorder) é um gravador acústico autônomo fabricado pela empresa JASCO Applied
Sciences.

O equipamento tem as seguintes características:
* Possibilidade de gravação agendada;
* Capacidade de até 1.792 GBytes de gravação;
* Até 8 canais de 24 bits com frequência de digitalização até 128
kHz;
* Um canal de 16 bits de alta velocidade, até 687,5 kHz;
* Profundidade de operação até 250m, no modo Standard e até

2.500m no modo Deep;
* Dimensões de 16,5 cm de diâmetro e 56,2 cm de altura;
* Opções de 9, 48, 96 ou 144 baterias alcalinas tamanho D;
* Tempo de gravação de até um ano;
* Ganho ajustável de O a 42 dB;
* Dez canais para medições oceanográficas.

O gravador acústico é equipado com um hidrofone modelo M14-V20-370 ou M36-V35-100, da GeoSpectrum. 

O modelo M14-V20-370 possui as seguintes características:
* Faixa de frequência: 10 Hz a 32 kHz;
* Pré-amplificação embutida de 20 dB;
* Resposta de frequência plana até 40 kHz (+/- 2 dB).

Já o modelo M36-V35-100 tem as seguintes características:
* Faixa de frequência 25 Hz a 60 kHz;
* Pré-amplificação embutida de 35 dB
* Resposta de frequência plana até 30 kHz (+/- 2 dB).

# Extraindo informações do arquivo ".csv" com as campanhas

In [99]:
df_LF = pd.read_excel(campanha_csv)

In [100]:
df_LF

,name,ciclo,starttime,endtime,fs,nBits,sensor,preamp,registrador
0,FA01-050m,LFI-C01,2017-10-28 00:00:00,2018-06-12 00:00:00,64000,24,M14-C000802,6,AMAR165
1,FA01-200m,LFI-C01,2017-10-28 00:00:00,2018-06-12 00:00:00,64000,24,M14-C000812,6,AMAR092
2,FA01-950m,LFI-C01,2017-10-28 00:00:00,2018-06-12 00:00:00,64000,24,M14-C000813,6,AMAR115
3,FA02-050m,LFI-C01,2017-10-16 00:00:00,2018-04-12 00:00:00,64000,24,M14-C000807,6,AMAR045
4,FA02-200m,LFI-C01,2017-10-16 00:00:00,2018-04-12 00:00:00,64000,24,M14-C000809,6,AMAR096
...,...,...,...,...,...,...,...,...,...
141,FA04-950m,LFI-C10,2022-08-01 16:33:00,2022-10-21 17:20:00,64000,24,M14-C000805,18,AMAR116
142,FA05-200m,LFI-C10,2022-08-27 01:40:00,2022-10-14 10:30:00,64000,24,M14-D001053,18,AMAR045
143,FA05-950m,LFI-C10,2022-08-11 00:50:00,2022-10-18 18:47:00,64000,24,M14-C000804,18,AMAR187
144,FA06-200m,LFI-C10,2022-07-17 19:50:00,2022-10-21 17:20:00,64000,24,M14-D000911,18,AMAR184


In [101]:
df_LF['starttime'] = pd.to_datetime(df_LF.starttime, format='%Y-%m-%d %H:%M:%S').dt.date
df_LF['endtime'] = pd.to_datetime(df_LF.endtime, format='%Y-%m-%d %H:%M:%S').dt.date

In [116]:
df_LF['depth'] = df_LF['name'].str[5:-1]
df_LF['name_LFI'] = df_LF['name'].str[:4]

In [118]:
df_LF.to_feather('/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/OUTPUT/NETTAB/df_LFI_nettab.feather')

In [119]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [120]:
def convert_36(line):
       
    datex = UTCDateTime(line['starttime']).timestamp

    # date range between 2015-2030 quatrimesters:
    date_range = [i.timestamp() for i in pd.date_range(start='01/01/2015', end='31/12/2030', freq='4M')]
    
    # hexatrigesimal two digit number for each quatrimester:
    hexatri_lst = [str(np.base_repr(y, 36)).zfill(2) for y in range(len(date_range))]

    if line['depth'] == '050':
        #find the nearest date and append the 2-hexatrigesimal-digit:
        return line['name_LFI'][0]+line['name_LFI'][-1]+hexatri_lst[date_range.index(nearest(date_range,datex))]+'A'
    
    if line['depth'] == '200':
        #find the nearest date and append the 2-hexatrigesimal-digit:
       return line['name_LFI'][0]+line['name_LFI'][-1]+hexatri_lst[date_range.index(nearest(date_range,datex))]+'B'

    if line['depth'] == '950':
        #find the nearest date and append the 2-hexatrigesimal-digit:
        return line['name_LFI'][0]+line['name_LFI'][-1]+hexatri_lst[date_range.index(nearest(date_range,datex))]+'C'


In [121]:
df_LF['nettab_name'] = df_LF.apply(convert_36,axis=1)

In [122]:
df_LF['starttime'] = pd.to_datetime(df_LF.starttime, format='%d-%m-%Y')
df_LF['endtime'] = pd.to_datetime(df_LF.endtime, format='%d-%m-%Y')

In [123]:
df_LF['date_start_nettab'] = df_LF.starttime.dt.strftime('%Y/%j')
df_LF['date_end_nettab'] = df_LF.endtime.dt.strftime('%Y/%j')

In [124]:
df_LF

,name,ciclo,starttime,endtime,fs,nBits,sensor,preamp,registrador,depth,nettab_name,date_start_nettab,date_end_nettab,name_LFI
0,FA01-050m,LFI-C01,2017-10-28,2018-06-12,64000,24,M14-C000802,6,AMAR165,050,F108A,2017/301,2018/163,FA01
1,FA01-200m,LFI-C01,2017-10-28,2018-06-12,64000,24,M14-C000812,6,AMAR092,200,F108B,2017/301,2018/163,FA01
2,FA01-950m,LFI-C01,2017-10-28,2018-06-12,64000,24,M14-C000813,6,AMAR115,950,F108C,2017/301,2018/163,FA01
3,FA02-050m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000807,6,AMAR045,050,F208A,2017/289,2018/102,FA02
4,FA02-200m,LFI-C01,2017-10-16,2018-04-12,64000,24,M14-C000809,6,AMAR096,200,F208B,2017/289,2018/102,FA02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,FA04-950m,LFI-C10,2022-08-01,2022-10-21,64000,24,M14-C000805,18,AMAR116,950,F40NC,2022/213,2022/294,FA04
142,FA05-200m,LFI-C10,2022-08-27,2022-10-14,64000,24,M14-D001053,18,AMAR045,200,F50NB,2022/239,2022/287,FA05
143,FA05-950m,LFI-C10,2022-08-11,2022-10-18,64000,24,M14-C000804,18,AMAR187,950,F50NC,2022/223,2022/291,FA05
144,FA06-200m,LFI-C10,2022-07-17,2022-10-21,64000,24,M14-D000911,18,AMAR184,200,F60MB,2022/198,2022/294,FA06


In [112]:
lines_nettab = []
for i in df_LF.sort_values(by='name').iterrows():
    # Station lines (one line per station/sensor/epoch) # code description datalogger%sn seismometer%sn # sampling orientation lat. lon. elev. depth. start end
    #Sl: UNAP "Uni-Iquique/Chile"   DM24%A1383 CMG-3ESP/60%T34622 100 ZNE -20.24393 -70.14041 0.0    0.0 2009/134 
    print('Sl: '+i[1].nettab_name+' "LFI:'+i[1]['name']+'" '+i[1].registrador+' '+i[1].sensor+' 100 H '+"{:.4f}".format(i[1].latitude)+' '+"{:.4f}".format(i[1].longitude)+' '+str(i[1].depth)+' 0.0 '+i[1].date_start_nettab+' '+i[1].date_end_nettab)   
    lines_nettab.append('Sl: '+i[1].nettab_name+' LFI:'+str(i[1].name)+' '+i[1].registrador+' '+str(i[1].sensor)+' 100 Z '+"{:.4f}".format(i[1].latitude)+' '+"{:.4f}".format(i[1].longitude)+' '+str(i[1].depth)+' 0.0 '+i[1].date_start_nettab+' '+i[1].date_end_nettab)

AttributeError: 'Series' object has no attribute 'latitude'

In [155]:
with open('/media/sysop/8d2362fc-3b46-49a7-a864-19b2a6ad097b/diogoloc/dados_posdoc/gliders_project/gliders_data/info_csv/df_LFI_nettab.txt', 'w') as f:
    for line in lines_nettab:
        f.write(line)
        f.write('\n')

# Renomeando os arquivos ".mseed":

In [ ]:
def rename_files(k):
    
    files_mseed_HD = k[1]['filename_mseed_HD']
    
    files_path_mseed_HD = [glob.glob(MSEED_INPUT+'*/*/*'+i+'.mseed') for i in files_mseed_HD]
    for y in files_path_mseed_HD:
        
        st = obspy.read(y[0])
        st[0].stats.network = 'GL'
        st[0].stats.station = k[1]['station_nettab']
        st[0].stats.location = k[1]['mergulho_nettab']
        st[0].stats.channel = 'HHH'

        #<SDSdir>/Year/NET/STA/CHAN.TYPE/NET.STA.LOC.CHAN.TYPE.YEAR.DAY
        #Saving in SDS structure        
        OUTPUT_STREAM = FOLDER_OUTPUT+'MSEED_SDS/'+st[0].stats.starttime.strftime("%Y")+'/'+st[0].stats.network+'/'+st[0].stats.station+'/HHH.D/'
        os.makedirs(OUTPUT_STREAM,exist_ok=True)
        
        station_name_file = st[0].stats.network+'.'+st[0].stats.station+'..'+st[0].stats.channel+'.D.'+st[0].stats.starttime.strftime("%Y.%j")+'.'+y[0].split('/')[-1]
        if os.path.exists(OUTPUT_STREAM+station_name_file) == False:
            st.write(OUTPUT_STREAM+station_name_file, format='MSEED')

In [ ]:
with Pool(processes=4) as p:
    with tqdm(total=len(df_mergulho_feather)) as pbar:
        for result in p.imap_unordered(rename_files,dataframe_mseed_final.iterrows()):
            pbar.update()